In [7]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import MobileNetV2

In [11]:
# -------- USER CONFIG (change paths) --------
TRAIN_DIR = r"C:\Users\racha\Downloads\caltech-101-img\caltech-101-img\train"
VAL_DIR   = r"C:\Users\racha\Downloads\caltech-101-img\caltech-101-img\val"
IMG_SIZE  = (128, 128)
BATCH     = 32
EPOCHS_HEAD = 3
EPOCHS_FINE = 1
SAVE_NAME = "fast_mobilenet_caltech"

In [5]:
# sanity
if not os.path.isdir(TRAIN_DIR): raise ValueError("TRAIN_DIR not found")
if not os.path.isdir(VAL_DIR): raise ValueError("VAL_DIR not found")

In [12]:
# --- create the 'classes' list (IMPORTANT) ---
classes = sorted([d for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))])
NUM_CLASSES = len(classes)
print("Detected classes:", NUM_CLASSES)
print("Example:", classes[:5])

Detected classes: 102
Example: ['BACKGROUND_Google', 'Faces', 'Faces_easy', 'Leopards', 'Motorbikes']


In [13]:
# --- create datasets ---
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    labels='inferred',
    label_mode='categorical',
    class_names=classes,         # use same class order for both sets
    image_size=IMG_SIZE,
    batch_size=BATCH,
    shuffle=True
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_DIR,
    labels='inferred',
    label_mode='categorical',
    class_names=classes,
    image_size=IMG_SIZE,
    batch_size=BATCH,
    shuffle=False
)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(AUTOTUNE)
val_ds   = val_ds.cache().prefetch(AUTOTUNE)


Found 7280 files belonging to 102 classes.
Found 1864 files belonging to 102 classes.


In [14]:
# --- build small fast model (MobileNetV2 backbone) ---
base = MobileNetV2(input_shape=(*IMG_SIZE,3), include_top=False, weights='imagenet')
base.trainable = False

inputs = layers.Input(shape=(*IMG_SIZE,3))
x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = models.Model(inputs, outputs)
model.compile(optimizer=optimizers.Adam(1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

9406464/9406464 [==============================] - 1s 0us/step


In [15]:
# --- train classifier head (quick) ---
model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_HEAD)

Epoch 1/3
228/228 [==============================] - 224s 923ms/step - loss: 1.8325 - accuracy: 0.6120 - val_loss: 0.5679 - val_accuracy: 0.8632
Epoch 2/3
228/228 [==============================] - 204s 895ms/step - loss: 0.4915 - accuracy: 0.8727 - val_loss: 0.4294 - val_accuracy: 0.8841
Epoch 3/3
228/228 [==============================] - 216s 949ms/step - loss: 0.2804 - accuracy: 0.9240 - val_loss: 0.3880 - val_accuracy: 0.8906


In [16]:
# --- quick fine-tune: unfreeze last few layers ---
for layer in base.layers[:-15]:
    layer.trainable = False
for layer in base.layers[-15:]:
    layer.trainable = True

model.compile(optimizer=optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_FINE)

228/228 [==============================] - 243s 990ms/step - loss: 0.1808 - accuracy: 0.9544 - val_loss: 0.3741 - val_accuracy: 0.8981
